In [5]:
! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 

In [3]:
import langchain
print(langchain.__version__)

0.3.26


In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-4o-mini")
chain = llm | output_parser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

In [ ]:
# 1. Function to load documents from a folder
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Load documents from a folder
folder_path = "/content/docs"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 4 documents from the folder.
Split the documents into 5 chunks.


In [ ]:
embeddings = OpenAIEmbeddings()

# 4. Embedding Documents

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 5 document chunks.


In [ ]:
document_embeddings[0]

[0.002419416094198823,
 -0.0007589008309878409,
 -0.007971816696226597,
 -0.006094712298363447,
 -0.007333802524954081,
 0.028314387425780296,
 -0.026272743940353394,
 -0.017098775133490562,
 -0.002587314462289214,
 -0.004983224906027317,
 0.01912698708474636,
 -0.01571529172360897,
 -0.002610820345580578,
 -0.005873086396604776,
 0.016830136999487877,
 -0.008052407763898373,
 0.014278081245720387,
 -0.048435334116220474,
 0.008401636965572834,
 -0.011061147786676884,
 -0.005758915562182665,
 0.03073212504386902,
 0.015379494056105614,
 0.01179990079253912,
 0.003908675163984299,
 0.0014212600653991103,
 -0.009523198008537292,
 -0.005339169409126043,
 -0.016642089933156967,
 -0.004261261783540249,
 -0.00220282725058496,
 -0.00815314706414938,
 -0.009073230437934399,
 0.01762261800467968,
 -0.011356648989021778,
 -0.002773681655526161,
 -0.011618570424616337,
 0.00016223183774854988,
 0.004633996170014143,
 -0.02616528794169426,
 0.028341252356767654,
 0.014989970251917839,
 -0.00209201

In [ ]:
# !pip install sentence_transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
document_embeddings[0]

/tmp/ipython-input-15-3719339062.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.04015983268618584,
 -0.0502498522400856,
 -0.02303307130932808,
 0.025236131623387337,
 -0.07738399505615234,
 -0.05450466647744179,
 -0.006515617948025465,
 -0.03506862744688988,
 -0.020189901813864708,
 -0.058069415390491486,
 -0.04932286590337753,
 -0.08118970692157745,
 0.11538005620241165,
 0.007650453131645918,
 -0.006956446450203657,
 -0.01566854864358902,
 0.014845970086753368,
 -0.08999183028936386,
 0.09548774361610413,
 -0.06116339564323425,
 0.0032092700712382793,
 -0.03143874928355217,
 0.017659202218055725,
 -0.08417436480522156,
 0.05682242661714554,
 -0.015141190029680729,
 0.016150420531630516,
 -0.06000141799449921,
 0.007828919216990471,
 -0.0707516148686409,
 0.03501879796385765,
 0.1001533791422844,
 -0.009450633078813553,
 -0.03534579649567604,
 0.01530828420072794,
 0.018368098884820938,
 -0.03543583303689957,
 -0.011511540040373802,
 0.0635034590959549,
 -0.06266675889492035,
 -0.010365902446210384,
 0.02724468894302845,
 -0.06460905075073242,
 0.027788156643

In [ ]:
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [ ]:
# 5. Perform similarity search

query = "When was GreenGrow Innovations founded?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'When was GreenGrow Innovations founded?'

Result 1:
Source: /content/docs/GreenGrow Innovations_ Company History.docx
Content: The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.



Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.



Despite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("When was GreenGrow Innovations founded?")

[Document(id='3ee6357e-05d2-40da-a340-4b3cd99f38e4', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research inst

In [ ]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(response)

GreenGrow Innovations was founded in 2010.


###Conversational RAG

####Handling Follow Up Questions

In [ ]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [ ]:
chat_history

[HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='GreenGrow Innovations was founded in 2010.', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
# contextualize_chain.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

In [ ]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

[Document(id='3ee6357e-05d2-40da-a340-4b3cd99f38e4', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research inst

In [ ]:
retriever.invoke("Where it is headquartered?")

[Document(id='7d6f8fdb-b7fa-415e-8ddb-6de1ea801db4', metadata={'source': '/content/docs/Company_ QuantumNext Systems.docx'}, page_content='Company: QuantumNext Systems\n\nHeadquarters: QuantumNext Systems is headquartered in Bangalore, Karnataka, India. The company, specializing in quantum computing and advanced data processing, is situated in the bustling tech metropolis of Bangalore, often referred to as the "Silicon Valley of India." From this technology capital, QuantumNext Systems is well-positioned to tap into India\'s rich pool of engineering talent and growing tech ecosystem, enabling it to push the boundaries of computational innovation.'),
 Document(id='f914a29f-4f10-4192-98a7-553235a25ab8', metadata={'source': '/content/docs/Company_ TechWave Innovations.docx'}, page_content='Company: TechWave Innovations\n\nHeadquarters: TechWave Innovations is headquartered in San Francisco, California, USA. As a leader in cutting-edge AI and machine learning solutions, the company thrives

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    #  ("system", "Tell me joke on Programming"),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
rag_chain.invoke({"input": "Where it is headquartered?", "chat_history":chat_history})

{'input': 'Where it is headquartered?',
 'chat_history': [HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='GreenGrow Innovations was founded in 2010.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='3ee6357e-05d2-40da-a340-4b3cd99f38e4', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing proj

###Building Multi User Chatbot

In [ ]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [ ]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: When was GreenGrow Innovations founded?
AI: GreenGrow Innovations was founded in 2010.



In [ ]:
question2 = "Where it is headquartered?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'When was GreenGrow Innovations founded?'}, {'role': 'ai', 'content': 'GreenGrow Innovations was founded in 2010.'}]
Human: Where it is headquartered?
AI: GreenGrow Innovations is headquartered in Portland, Oregon.



New User

In [ ]:
session_id = str(uuid.uuid4())
question = "What is GreenGrow"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is GreenGrow
AI: GreenGrow Innovations is a company focused on developing sustainable agricultural technologies. Founded in 2010 by Sarah Chen and Michael Rodriguez in Portland, Oregon, GreenGrow aims to make farming more environmentally friendly and efficient. The company initially started with smart irrigation systems and has grown to create various products, including the WaterWise Sensor and the SoilHealth Monitor. Their notable breakthrough was the EcoHarvest System, which integrates smart irrigation, soil monitoring, and automated harvesting techniques. As of now, GreenGrow employs over 200 people, has expanded its operations to other states, and continues to work on innovative solutions for sustainable farming practices.

